In [3]:
import sys
import os
from pathlib import Path

# Caminho absoluto para a raiz do projeto (um nível acima de notebooks)
project_root = os.path.abspath("..")
if project_root not in sys.path:
    sys.path.append(project_root)


In [4]:
from config.utils import *


In [ ]:
PATHS['BRONZE_LAYER']

'C:\\Users\\BROliveiWa12\\OneDrive - NESTLE\\Documents\\Estudos\\Engenharia de Dados\\economic-indicators-pipeline\\data\\bronze'

In [13]:
teste = obter_dados_api(INDICADORES['IPCA'])

2025-07-01 23:29:03.664 | INFO     | __main__:obter_dados_api:16 - Realizando requisição da API: Índice Nacional de Preços ao Consumidor Amplo (IPCA)
2025-07-01 23:29:03.781 | SUCCESS  | __main__:obter_dados_api:19 - Requisição para Índice Nacional de Preços ao Consumidor Amplo (IPCA) realizada com sucesso - Status: 200


In [14]:
#INDICADORES['IPCA']['URL'],
INDICADORES['DESEMPREGO']['URL']

'https://servicodados.ibge.gov.br/api/v3/agregados/4099/periodos/all/variaveis/4099?localidades=N1[all]'

In [21]:
os.path.join(PATHS["BRONZE_LAYER"],'teste.json')

'C:\\Users\\BROliveiWa12\\OneDrive - NESTLE\\Documents\\Estudos\\Engenharia de Dados\\economic-indicators-pipeline\\data\\bronze\\teste.json'

In [22]:
teste[0]['resultados'][0]['series'][0]['serie']

{'202001': '0.21',
 '202002': '0.25',
 '202003': '0.07',
 '202004': '-0.31',
 '202005': '-0.38',
 '202006': '0.26',
 '202007': '0.36',
 '202008': '0.24',
 '202009': '0.64',
 '202010': '0.86',
 '202011': '0.89',
 '202012': '1.35',
 '202101': '0.25',
 '202102': '0.86',
 '202103': '0.93',
 '202104': '0.31',
 '202105': '0.83',
 '202106': '0.53',
 '202107': '0.96',
 '202108': '0.87',
 '202109': '1.16',
 '202110': '1.25',
 '202111': '0.95',
 '202112': '0.73',
 '202201': '0.54',
 '202202': '1.01',
 '202203': '1.62',
 '202204': '1.06',
 '202205': '0.47',
 '202206': '0.67',
 '202207': '-0.68',
 '202208': '-0.36',
 '202209': '-0.29',
 '202210': '0.59',
 '202211': '0.41',
 '202212': '0.62',
 '202301': '0.53',
 '202302': '0.84',
 '202303': '0.71',
 '202304': '0.61',
 '202305': '0.23',
 '202306': '-0.08',
 '202307': '0.12',
 '202308': '0.23',
 '202309': '0.26',
 '202310': '0.24',
 '202311': '0.28',
 '202312': '0.56',
 '202401': '0.42',
 '202402': '0.83',
 '202403': '0.16',
 '202404': '0.38',
 '2024

In [23]:
salvar_dados_api(teste, 'ipca')

2025-07-01 23:30:42.316 | INFO     | __main__:salvar_dados_api:20 - Indicacdor ipca salvos no arquivo: ipca_20250701.json


2025-07-01 23:30:42.490 | INFO     | __main__:obter_dados_api:16 - Realizando requisição da API: Índice Nacional de Preços ao Consumidor Amplo (IPCA)
2025-07-01 23:30:42.663 | SUCCESS  | __main__:obter_dados_api:19 - Requisição para Índice Nacional de Preços ao Consumidor Amplo (IPCA) realizada com sucesso - Status: 200
2025-07-01 23:30:42.663 | INFO     | __main__:salvar_dados_api:20 - Indicacdor IPCA salvos no arquivo: ipca_20250701.json
2025-07-01 23:30:42.663 | INFO     | __main__:obter_dados_api:16 - Realizando requisição da API: Taxa Selic Meta
2025-07-01 23:30:43.001 | SUCCESS  | __main__:obter_dados_api:19 - Requisição para Taxa Selic Meta realizada com sucesso - Status: 200
2025-07-01 23:30:43.029 | INFO     | __main__:salvar_dados_api:20 - Indicacdor SELIC_META salvos no arquivo: selic_meta_20250701.json
2025-07-01 23:30:43.029 | INFO     | __main__:obter_dados_api:16 - Realizando requisição da API: Índice de Atividade Econômica do Banco Central (IBC-Br)
2025-07-01 23:30:43.2

In [ ]:
for nome_indicador, config in INDICADORES.values():
    try:
        dados = obter_dados_api(indicador)
        salvar_dados_api(dados, indicador['NOME'])
    except Exception as e:
        logger.error(f"Erro ao processar o indicador {indicador['NOME']}: {str(e)}")

In [ ]:

Cotaçao cambio
response = requests.get("https://api.bcb.gov.br/dados/serie/bcdata.sgs.21619/dados?formato=json&dataInicial=29/06/2015")
response.json()

[{'data': '29/06/2015', 'valor': '3.5094000'},
 {'data': '30/06/2015', 'valor': '3.4603000'},
 {'data': '01/07/2015', 'valor': '3.4569000'},
 {'data': '02/07/2015', 'valor': '3.4577000'},
 {'data': '03/07/2015', 'valor': '3.4701000'},
 {'data': '06/07/2015', 'valor': '3.4813000'},
 {'data': '07/07/2015', 'valor': '3.4840000'},
 {'data': '08/07/2015', 'valor': '3.5743000'},
 {'data': '09/07/2015', 'valor': '3.5460000'},
 {'data': '10/07/2015', 'valor': '3.5568000'},
 {'data': '13/07/2015', 'valor': '3.4823000'},
 {'data': '14/07/2015', 'valor': '3.4403000'},
 {'data': '15/07/2015', 'valor': '3.4575000'},
 {'data': '16/07/2015', 'valor': '3.4192000'},
 {'data': '17/07/2015', 'valor': '3.4535000'},
 {'data': '20/07/2015', 'valor': '3.4882000'},
 {'data': '21/07/2015', 'valor': '3.4762000'},
 {'data': '22/07/2015', 'valor': '3.4934000'},
 {'data': '23/07/2015', 'valor': '3.6053000'},
 {'data': '24/07/2015', 'valor': '3.6503000'},
 {'data': '27/07/2015', 'valor': '3.7295000'},
 {'data': '28

In [10]:
url = "https://api.bcb.gov.br/dados/serie/bcdata.sgs.432/dados?formato=json&dataInicial=01/07/2015&dataFinal=01/07/2025"

response = requests.get(url)
response.raise_for_status()  # Verifica se a requisição foi bem-sucedida
dados_cambio = response.json()

In [11]:
dados_cambio

[{'data': '01/07/2015', 'valor': '13.75'},
 {'data': '02/07/2015', 'valor': '13.75'},
 {'data': '03/07/2015', 'valor': '13.75'},
 {'data': '04/07/2015', 'valor': '13.75'},
 {'data': '05/07/2015', 'valor': '13.75'},
 {'data': '06/07/2015', 'valor': '13.75'},
 {'data': '07/07/2015', 'valor': '13.75'},
 {'data': '08/07/2015', 'valor': '13.75'},
 {'data': '09/07/2015', 'valor': '13.75'},
 {'data': '10/07/2015', 'valor': '13.75'},
 {'data': '11/07/2015', 'valor': '13.75'},
 {'data': '12/07/2015', 'valor': '13.75'},
 {'data': '13/07/2015', 'valor': '13.75'},
 {'data': '14/07/2015', 'valor': '13.75'},
 {'data': '15/07/2015', 'valor': '13.75'},
 {'data': '16/07/2015', 'valor': '13.75'},
 {'data': '17/07/2015', 'valor': '13.75'},
 {'data': '18/07/2015', 'valor': '13.75'},
 {'data': '19/07/2015', 'valor': '13.75'},
 {'data': '20/07/2015', 'valor': '13.75'},
 {'data': '21/07/2015', 'valor': '13.75'},
 {'data': '22/07/2015', 'valor': '13.75'},
 {'data': '23/07/2015', 'valor': '13.75'},
 {'data': '